In [ ]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
## connecting to google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/Dataset/"

# Create YOLOv8 directory
yolov8_path = os.path.join(dataset_path, "yolov8")
os.makedirs(yolov8_path, exist_ok=True)

# Create train, test, and valid directories inside yolov8
sets = ["train", "test", "valid"]
for set_type in sets:
    set_path = os.path.join(yolov8_path, set_type)
    os.makedirs(os.path.join(set_path, "images"), exist_ok=True)
    os.makedirs(os.path.join(set_path, "labels"), exist_ok=True)

# Get the list of image files
image_files = os.listdir(os.path.join(dataset_path, "Images"))

# Split the data into train, test, and valid sets (70:15:15)
train_files, test_valid_files = train_test_split(image_files, test_size=0.3, random_state=42)
test_files, valid_files = train_test_split(test_valid_files, test_size=0.5, random_state=42)

# Move images and labels to the respective YOLOv8 folders
def move_files(file_list, set_type):
    for file in file_list:
        # Move images
        image_src = os.path.join(dataset_path, "Images", file)
        image_dest = os.path.join(yolov8_path, set_type, "images", file)
        shutil.copy(image_src, image_dest)

        # Move labels
        label_file = file.replace(".jpg", ".txt")
        label_src = os.path.join(dataset_path, "Labels", label_file)
        label_dest = os.path.join(yolov8_path, set_type, "labels", label_file)
        shutil.copy(label_src, label_dest)

# Move files to train set
move_files(train_files, "train")

# Move files to test set
move_files(test_files, "test")

# Move files to valid set
move_files(valid_files, "valid")


In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/Dataset/yolov8/data/custom.yaml epochs=50 imgsz=640

100% 6.23M/6.23M [00:00<00:00, 62.9MB/s]
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Dataset/yolov8/data/custom.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

In [ ]:
!yolo task=detect mode=val model=yolov8n.pt data=/content/drive/MyDrive/Dataset/yolov8/data/dataset.yaml

# loading the model version - yolov8n.pt
# and the path for dataset.yaml which let's know the model
# about the paths of test, training and validation.

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /content/drive/MyDrive/Dataset/yolov8/valid/labels.cache... 29 images, 0 backgrounds, 0 corrupt: 100% 29/29 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:11<00:00,  5.54s/it]
                   all         29         29     0.0436      0.195     0.0321     0.0156
                person         29         12     0.0872       0.39     0.0539     0.0291
               bicycle         29         17          0          0     0.0104    0.00208
Speed: 11.4ms preprocess, 332.4ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [ ]:
!yolo task=detect mode=test model=yolov8n.pt data=/content/drive/MyDrive/Dataset/yolov8/data/custom.yaml

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 392, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=test'. Valid modes are ('train', 'val', 'predict', 'export', 'track', 'benchmark').

    Arguments received: ['yolo', 'task=detect', 'mode=test', 'model=yolov8n.pt', 'data=/content/drive/MyDrive/Dataset/yolov8/data/custom.yaml']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultral

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.3 source=/content/drive/MyDrive/Dataset/yolov8/test/images

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

image 1/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_19.jpg: 352x640 (no detections), 288.9ms
image 2/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_22.jpg: 352x640 (no detections), 215.9ms
image 3/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_25.jpg: 352x640 (no detections), 205.9ms
image 4/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_26.jpg: 352x640 (no detections), 206.7ms
image 5/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_27.jpg: 352x640 (no detections), 212.2ms
image 6/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_30.jpg: 352x640 (no detections), 213.5ms
image 7/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_33.jpg: 512x640 1 handgun, 318.4ms
image 8/28 /content/drive/MyDrive/Dataset/yolov8/test/images/handgun_38

In [ ]:
!yolo val model=/content/runs/detect/train/weights/best.pt data=/content/drive/MyDrive/Dataset/yolov8/data/custom.yaml

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/Dataset/yolov8/valid/labels.cache... 29 images, 0 backgrounds, 0 corrupt: 100% 29/29 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:13<00:00,  6.80s/it]
                   all         29         29      0.761       0.69      0.655      0.475
               handgun         29         12      0.651      0.583      0.533      0.428
                 knife         29         17      0.871      0.797      0.778      0.521
Speed: 14.8ms preprocess, 426.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val
